In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plots
sns.set(style="whitegrid")


In [ ]:
# Step 2: Load the dataset from your data folder
# Make sure the file is named customer_subscription_data.csv
df = pd.read_csv("../data/customer_subscription_data.csv")

# Preview first 5 rows
df.head()



In [ ]:
# Step 3: Explore dataset structure
print("Dataset Info:")
df.info()

print("\nMissing Values:")
print(df.isnull().sum())

print("\nSummary Statistics:")
print(df.describe())

# Look at first 10 rows
df.head(10)


In [ ]:
# Step 4: Calculate churn rate (assuming column 'Churned' has Yes/No values)
if 'Churned' in df.columns:
    churn_rate = df['Churned'].value_counts(normalize=True)['Yes']
    print(f"Churn Rate: {churn_rate:.2%}")
else:
    print("No 'Churned' column found. Please check dataset columns.")


In [ ]:
# Step 5: Visualize churn vs retained customers
if 'Churned' in df.columns:
    sns.countplot(x="Churned", data=df, palette="Set2")
    plt.title("Churn vs Retained Customers")
    plt.show()


In [ ]:
# Step 6: Cohort analysis - group by signup month
if 'StartDate' in df.columns:
    df['SignupMonth'] = pd.to_datetime(df['StartDate']).dt.to_period('M')
    cohort = df.groupby(['SignupMonth','Churned']).size().unstack(fill_value=0)
    print("\nCohort Retention Table:")
    print(cohort)
else:
    print("No 'StartDate' column found. Please check dataset columns.")


In [ ]:
# Step 7: Plot retention trend over time
if 'SignupMonth' in df.columns and 'Churned' in df.columns:
    retention = df.groupby('SignupMonth')['Churned'].apply(lambda x: (x=='No').mean())
    retention.plot(kind='line', marker='o', figsize=(10,5))
    plt.title("Retention Trend by Signup Month")
    plt.ylabel("Retention Rate")
    plt.xlabel("Signup Month")
    plt.show()


In [ ]:
# Step 6: Cohort Analysis - Retention by Signup Month

# Convert StartDate to datetime
df['StartDate'] = pd.to_datetime(df['StartDate'])

# Create a column for signup month
df['SignupMonth'] = df['StartDate'].dt.to_period('M')

# Create a retention table: how many churned vs retained per signup month
cohort = df.groupby(['SignupMonth','Churned']).size().unstack(fill_value=0)

print("Cohort Retention Table:")
print(cohort)


In [ ]:
# Step 7: Retention Trend Visualization

# Calculate retention rate per signup month
retention = df.groupby('SignupMonth')['Churned'].apply(lambda x: (x=='No').mean())

# Plot retention trend
retention.plot(kind='line', marker='o', figsize=(10,5))
plt.title("Retention Trend by Signup Month")
plt.ylabel("Retention Rate")
plt.xlabel("Signup Month")
plt.show()


In [ ]:
# Step 8: Customer Lifetime Value (CLV)

from lifetimes import BetaGeoFitter, GammaGammaFitter

# Prepare data: frequency, recency, and age of customers
# Assumes you have columns: CustomerID, StartDate, EndDate, Revenue
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])

# Calculate customer metrics
customer_metrics = df.groupby('CustomerID').agg({
    'StartDate': lambda x: (x.max() - x.min()).days,   # customer age
    'EndDate': 'count',                               # frequency of renewals
    'Revenue': 'sum'                                  # total revenue
}).reset_index()

customer_metrics.columns = ['CustomerID', 'CustomerAge', 'Frequency', 'MonetaryValue']

# Fit BG/NBD model (for purchase frequency)
bgf = BetaGeoFitter()
bgf.fit(customer_metrics['Frequency'], customer_metrics['CustomerAge'], customer_metrics['CustomerAge'])

# Predict expected transactions in next 6 months
customer_metrics['PredictedPurchases'] = bgf.predict(180, customer_metrics['Frequency'], customer_metrics['CustomerAge'], customer_metrics['CustomerAge'])

# Fit Gamma-Gamma model (for monetary value)
ggf = GammaGammaFitter()
ggf.fit(customer_metrics['Frequency'], customer_metrics['MonetaryValue'])

# Estimate CLV
customer_metrics['CLV'] = ggf.customer_lifetime_value(
    bgf,
    customer_metrics['Frequency'],
    customer_metrics['CustomerAge'],
    customer_metrics['MonetaryValue'],
    time=12,  # months
    discount_rate=0.01
)

customer_metrics.head()


In [ ]:
# Step 9: Export results for dashboard
customer_metrics.to_csv("../data/churn_results.csv", index=False)
